# Final Draft Pick Model 

This notebook uses a neural network to predict what pick bin a player will be in with data consisting of NBA caliber players who have been drafted and not drafted but still played in NBA

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.formula.api import glm
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LassoCV
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import imblearn
from imblearn.over_sampling import SMOTE
from collections import Counter
from matplotlib import pyplot
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import keras
from sklearn.utils import class_weight
import keras
from keras.models import Sequential
from keras.layers import Dense

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# reading in the data 
roster = pd.read_csv("Roster Decode.csv")
pick = pd.read_csv("NBA Caliber Players.csv")

In [ ]:
seasons = pick[['PLAYER_ID', 'START_SEASON']]

In [ ]:
## grabbing only the columns needed 
df = roster[['PLAYER_ID', 'HEIGHT','WEIGHT', 'BIRTH_DATE', 'POSITION_ABBREV']]

In [ ]:
print(len(df))
print(len(roster))

In [ ]:
df = pd.merge(df, seasons, how='inner', on='PLAYER_ID')

In [ ]:
df['BIRTH_DATE'].isna().sum()

In [ ]:
## splitting the birth_date
df['YEAR'] = df['BIRTH_DATE'].str.split(pat = '-', expand = True)[2]

In [ ]:
df['YEAR'] = df['YEAR'].fillna(0)
df['START_SEASON'] = df['START_SEASON'].fillna(0)

In [ ]:
## converting column to an integer
df['YEAR'] = df['YEAR'].astype('int')

birth_date - july 1st of start year 

In [ ]:
## creating a list of years 
years = []
for i in df['YEAR']: 
    if i == 0: 
        years.append(i)
    elif i > 23: 
        years.append(i+1900) 
    elif i <= 23: 
        years.append(i+2000)

In [ ]:
# adding this list into the dataframe
df['YEAR'] = years

In [ ]:
df['Age'] = df['START_SEASON'] - df['YEAR']

In [ ]:
a = df[(df['Age'] < 57) & (df['Age'] > 0)]
a['Age'].mean()

In [ ]:
## filling in Age NAs
df.loc[df["Age"] > 56, "Age"] = a['Age'].mean()
df.loc[df['Age'] <= 0, "Age"] = a['Age'].mean()

In [ ]:
## getting the age of each player based on year 

# import pandas as pd
# from datetime import datetime, date
  
# # This function converts given date to age
# def age(born):
#     born = datetime.strptime(born, "%Y").date()
#     today = date.today()
#     return today.year - born.year 

In [ ]:
# ## appling the function
# df['Age'] = df['YEAR'].apply(age)

In [ ]:
## dropping unneeded columns
df.drop(['BIRTH_DATE', 'YEAR'], axis=1, inplace=True)

In [ ]:
## dropping duplicate players
df = df.drop_duplicates(subset='PLAYER_ID', keep="first")

In [ ]:
## drop people who started before 2012
df = df[df['START_SEASON'] >= 2012]

In [ ]:
df.rename({'POSITION_ABBREV': 'Pos'}, axis=1, inplace=True)

In [ ]:
df['Pos']=np.where(df['Pos']=='G','Guard',
                                    np.where(df['Pos']=='PG','Guard',
                                    np.where(df['Pos']=='SF','Wing',   
                                    np.where(df['Pos']=='PF','Big',         
                                    np.where(df['Pos']=='G/F','Wing',
                                    np.where(df['Pos']=='F',
                                    np.where(df['HEIGHT']<=79,'Wing','Big'),
                                    np.where(df['Pos']=='SG',
                                    np.where(df['HEIGHT']<=75,'Guard','Wing'),          
                                    np.where(df['Pos']=='F/C','Big',
                                    np.where(df['Pos']=='C','Big',
                                    np.where(df['HEIGHT']<=75,'Guard',
                                    np.where(df['HEIGHT']>79,'Big','Wing')))))))))))

In [ ]:
## Finding averages to replace NAs
df.groupby('Pos').agg({'HEIGHT': 'mean', 'WEIGHT': 'mean'})

In [ ]:
## replacing Height and Weight NAs
df['HEIGHT'] = df.groupby('Pos')['HEIGHT'].apply(lambda x: x.fillna(x.mean()))
df['WEIGHT'] = df.groupby('Pos')['WEIGHT'].apply(lambda x: x.fillna(x.mean()))


In [ ]:
## reading other data in 
data = pd.read_csv("incorporated_data.csv").drop(['Unnamed: 0'], axis=1)

In [ ]:
## dropping duplicates & replacing NAs
pick = pick.fillna(70)

In [ ]:
## making sure it is dropping the right duplicates
print(len(data))
print(len(pick))
print(len(df))

In [ ]:
## drop columns we don't need to merge in 
df.drop(['Pos','START_SEASON'], axis=1, inplace=True)
#data.drop(['AutoStats Games'], axis=1, inplace=True)

In [ ]:
## merging the biography data and the autostats data 
data = pd.merge(data, df, how='left', on='PLAYER_ID')

In [ ]:
## merging pick data 
data = pd.merge(data, pick, how='inner', on='PLAYER_ID')

In [ ]:
final_data = data.drop(['PICK_NUMBER_x'], axis=1)

In [ ]:
final_data.rename({'PICK_NUMBER_y':'PICK_NUMBER'}, axis=1, inplace=True)

In [ ]:
final_data.to_csv('final_data.csv')

In [ ]:
## replacing string with an integer to work with later 
# data['PICK_NUMBER_y'].replace('Other- NBA', 70.0, inplace=True)

# Box Stuff

In [ ]:
pick = pd.read_csv("NBA Caliber Players.csv")
seasons = pick[['PLAYER_ID', 'START_SEASON']]

In [ ]:
data = pd.read_csv('incorporated_data-Box.csv').drop(['Unnamed: 0', 'Drafted', 'PLAYER_GAMES'], axis=1)

In [ ]:
data = pd.merge(seasons, data, on='PLAYER_ID', how='inner')

In [ ]:
## grabbing start_season
start = data['START_SEASON']

In [ ]:
# splits data into X and Y for Bins told to do
X = data.drop(['PLAYER_ID', 'PICK_NUMBER', 'START_SEASON'], axis=1)
y = data[['PICK_NUMBER']]

# Normalize the data to be between -1 and 1 (mean centered)
X = (X-X.mean())/(X.max() - X.min())

### Bins I created 

In [ ]:
# splitting with new data
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify = y)

# y_train = y_train.astype('string')
# y_test = y_test.astype('string')

In [ ]:
X['START_SEASON'] = start

In [ ]:
## splitting the data based on season 
X_test = X[X['START_SEASON'] >= 2022]
y_test = y.loc[X_test.index]

X_train = X[(X['START_SEASON'] < 2022)]
y_train = y.loc[X_train.index]

In [ ]:
X_test.drop(['START_SEASON'], axis=1, inplace =True) 
X_train.drop(['START_SEASON'], axis=1, inplace =True) 

#### Recursive Feature Embedding

In [ ]:
rfc = RandomForestClassifier()


RFE_model = RFECV(
    estimator = rfc, # the ML model
    scoring = 'accuracy',  
    cv=2)

RFE_model.fit(X_train, y_train)

In [ ]:
mask = RFE_model.get_support()
features = np.array(X_train.columns) 
best_features = features[mask]
 
print("All features: ", X_train.shape[1])

print("Selected best: ", best_features.shape[0])

Take out anything that isn't a percent or / and anything with rank

anything that is a count (integer)

In [ ]:
for i in features[mask]:
    print("'" + i + "',")

In [ ]:
cols = ['FGM_30',
'FGA_30',
'TPA_30',
'FTM_30',
'FTA_30',
'FOULS_30',
'BLOCKS_30',
'STEALS_30',
'ASSISTS_30',
'DREB_30',
'TOV_30',
'FG_PCT',
'FT_PCT',
'USG',
'ASSIST_PCT',
'BLOCK_PCT',
'DREB_PCT',
'FTM_Rate',
'STL_PCT',
'TOV_PCT',
'undraft_prob']

In [ ]:
X_train = X_train[cols]
X_test = X_test[cols]

In [ ]:
# Convert X values to numpy array 
## These are already arrays
X_train_array = np.array(X_train)
X_test_array = np.array(X_test)

# encode labels as integers for y_train 
encoder = LabelEncoder()
encoder.fit(y_train)
encoder.classes_ = np.array(['1-4', '5-8','9-12', '13-18','19-30', '31-60', 'Undrafted' ])
encoded_Y_train = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_train = np_utils.to_categorical(encoded_Y_train)

# encode labels as integers for y_test 
encoder = LabelEncoder()
encoder.fit(y_test)
encoder.classes_ = np.array(['1-4', '5-8','9-12', '13-18','19-30', '31-60', 'Undrafted' ])
encoded_Y_test = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_test = np_utils.to_categorical(encoded_Y_test)

list(encoder.classes_)

In [ ]:
class_w = class_weight.compute_class_weight('balanced',
                                                 classes = ['1-4', '5-8', '9-12', '13-18', '19-30', '31-60', 'Undrafted'],
                                                 y = y_train.values.reshape(-1))
class_w

In [ ]:
for i, index in zip(class_w, range(0,10)): 
    r = print(index,":",i, ",")

In [ ]:
class_weights = {0 : 7.982142857142857 ,
1 : 8.514285714285714 ,
2 : 7.740259740259741 ,
3 : 5.1085714285714285 ,
4 : 2.528995756718529 ,
5 : 1.1352380952380952 ,
6 : 0.19394728278555157}

In [ ]:
# Neural network
model = Sequential()
model.add(Dense(16, input_dim=len(X_train.columns), activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train_array, dummy_y_train, 
                      validation_data = (X_test_array, dummy_y_test), 
                      epochs=100, 
                      class_weight = class_weights, 
                      batch_size=32)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
pred = model.predict(X_train_array)

predictions = pred.argmax(axis=1)

cm = confusion_matrix(encoded_Y_train, predictions)

In [ ]:
ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['1-4', '5-8', '9-12', '13-18', '19-30', '31-60', 'Other- NBA']); ax.yaxis.set_ticklabels(['1-4', '5-8', '9-12', '13-18', '19-30', '31-60', 'Other- NBA']);

In [ ]:
print(classification_report(encoded_Y_train, predictions))

#### CSV of Probabilities for Train (My Bins) 

In [ ]:
## getting indexs of the players then grabbing those IDs
index = list(X_train.index)
player_ids = data.loc[index, ['PLAYER_ID','START_SEASON','PICK_NUMBER']]

In [ ]:
player_ids = player_ids.reset_index(drop=True)

In [ ]:
## creating a dataframe of the probabilities of those players 
train_probs_other = pd.DataFrame(pred, columns = ['1-4', '5-8', '9-12', '13-18', '19-30', '31-60', 'Undrafted'])
result3 = pd.concat([player_ids, train_probs_other], axis=1)

In [ ]:
result3.to_csv("prob_train_box.csv")

**TEST**

In [ ]:
pred = model.predict(X_test_array)

predictions = pred.argmax(axis=1)

cm = confusion_matrix(encoded_Y_test, predictions)

In [ ]:
ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['1-4', '5-8', '9-12', '13-18', '19-30', '31-60', 'Undrafted']); ax.yaxis.set_ticklabels(['1-4', '5-8', '9-12', '13-18', '19-30', '31-60', 'Other- NBA']);

In [ ]:
print(classification_report(encoded_Y_test, predictions))

#### CSV of Probabilities for Test (MyBins)

In [ ]:
## getting indexs of the players then grabbing those IDs
index = list(X_test.index)
player_ids = data.loc[index, ['PLAYER_ID','START_SEASON','PICK_NUMBER']]

In [ ]:
player_ids = player_ids.reset_index(drop=True)

In [ ]:
## creating a dataframe of the probabilities of those players 
train_probs_other = pd.DataFrame(pred, columns = ['1-4', '5-8', '9-12', '13-18', '19-30', '31-60', 'Undrafted'])
result4 = pd.concat([player_ids, train_probs_other], axis=1)

In [ ]:
result4.to_csv("prob_test_box.csv")

#### CSV of Probabilities Combined (My Bins) 

In [ ]:
r2 = pd.concat([result3, result4])
r2['PLAYER_ID'].value_counts()

In [ ]:
r2.to_csv("combined_prob_box.csv")

In [ ]:
com = pd.read_csv("combined_prob_box.csv") 

In [ ]:
com[com['PLAYER_ID'].isin([1373356, 1349630])]

In [ ]:
com[com['PLAYER_ID'].isin([1373356, 1349630])]